In [2]:
import gensim.downloader as api
model_name = 'word2vec-google-news-300'
# wv_path = api.load(model_name, return_path=True)

In [12]:
print(wv_path)

/home/bardanik/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [3]:
wv = api.load(model_name)

In [4]:
pairs = [
    ('London', 'Paris'),
    ('London', 'bicycle'),
    ('capital', 'London'),
    ('London', 'Moscow'),
    ('Paris', 'capital'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))


'London'	'Paris'	0.53
'London'	'bicycle'	0.08
'capital'	'London'	0.21
'London'	'Moscow'	0.43
'Paris'	'capital'	0.20


KeyedVectors<vector_size=300, 3000000 keys>


In [35]:
print(wv.similar_by_word("cat"))
print(wv.similar_by_key("cat"))

[('cats', 0.8099379539489746), ('dog', 0.760945737361908), ('kitten', 0.7464984655380249), ('feline', 0.7326233983039856), ('beagle', 0.7150582671165466), ('puppy', 0.7075453996658325), ('pup', 0.6934291124343872), ('pet', 0.6891531348228455), ('felines', 0.6755931377410889), ('chihuahua', 0.6709762215614319)]
[('cats', 0.8099379539489746), ('dog', 0.760945737361908), ('kitten', 0.7464984655380249), ('feline', 0.7326233983039856), ('beagle', 0.7150582671165466), ('puppy', 0.7075453996658325), ('pup', 0.6934291124343872), ('pet', 0.6891531348228455), ('felines', 0.6755931377410889), ('chihuahua', 0.6709762215614319)]


In [23]:
import inspect
import gensim
# print(inspect.getsource(wv.similarity))
# print(inspect.getsource(gensim.matutils.unitvec))
print(inspect.getsource(wv.cosine_similarities))

    @staticmethod
    def cosine_similarities(vector_1, vectors_all):
        """Compute cosine similarities between one vector and a set of other vectors.

        Parameters
        ----------
        vector_1 : numpy.ndarray
            Vector from which similarities are to be computed, expected shape (dim,).
        vectors_all : numpy.ndarray
            For each row in vectors_all, distance from vector_1 is computed, expected shape (num_vectors, dim).

        Returns
        -------
        numpy.ndarray
            Contains cosine distance between `vector_1` and each row in `vectors_all`, shape (num_vectors,).

        """
        norm = np.linalg.norm(vector_1)
        all_norms = np.linalg.norm(vectors_all, axis=1)
        dot_products = dot(vectors_all, vector_1)
        similarities = dot_products / (norm * all_norms)
        return similarities



In [37]:
import numpy as np
wv_1 = wv['cat']
wv_2 = wv['dog']

product = np.dot(wv_1, wv_2)

wv_1_norm = np.linalg.norm(wv_1)
wv_2_norm = np.linalg.norm(wv_2)

print(product)

normalized_product = np.dot(wv_1/wv_1_norm, wv_2/wv_2_norm)

print(wv.similarity('cat', 'dog'),'vs',normalized_product)

wv.cosine_similarities(wv_1, [wv_2]) # also computes cosine similarity

6.8908596
0.76094574 vs 0.7609458


array([0.7609457], dtype=float32)

Trying now to implement first snippet of algorithm:

In [59]:
from gensim.models import KeyedVectors
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import string
from pprint import pprint
import re

In [53]:

# Load pre-trained Word2Vec model. Here I'm using Google's model, which you can download from
# https://code.google.com/archive/p/word2vec/
# Please note that this model is 1.5GB in size
# model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

text = "Animals are multicellular eukaryotic organisms that form the biological kingdom Animalia. With few exceptions, animals consume organic material, breathe oxygen, are able to move, reproduce sexually, and grow from a hollow sphere of cells, the blastula, during embryonic development. Over 1.5 million living animal species have been described—of which around 1 million are insects—but it has been estimated there are over 7 million animal species in total. Animals range in size from 8.5 micrometres to 33.6 metres. They have complex interactions with each other and their environments, forming intricate food webs. The study of animals is called zoology."
keyword = "animal"
threshold = 0.5  # set your similarity threshold here


In [66]:

# Tokenize the text into sentences and then into words
sentences = sent_tokenize(text)
words_lists = [word_tokenize(sentence) for sentence in sentences]
pprint(words_lists)

[['Animals',
  'are',
  'multicellular',
  'eukaryotic',
  'organisms',
  'that',
  'form',
  'the',
  'biological',
  'kingdom',
  'Animalia',
  '.'],
 ['With',
  'few',
  'exceptions',
  ',',
  'animals',
  'consume',
  'organic',
  'material',
  ',',
  'breathe',
  'oxygen',
  ',',
  'are',
  'able',
  'to',
  'move',
  ',',
  'reproduce',
  'sexually',
  ',',
  'and',
  'grow',
  'from',
  'a',
  'hollow',
  'sphere',
  'of',
  'cells',
  ',',
  'the',
  'blastula',
  ',',
  'during',
  'embryonic',
  'development',
  '.'],
 ['Over',
  '1.5',
  'million',
  'living',
  'animal',
  'species',
  'have',
  'been',
  'described—of',
  'which',
  'around',
  '1',
  'million',
  'are',
  'insects—but',
  'it',
  'has',
  'been',
  'estimated',
  'there',
  'are',
  'over',
  '7',
  'million',
  'animal',
  'species',
  'in',
  'total',
  '.'],
 ['Animals',
  'range',
  'in',
  'size',
  'from',
  '8.5',
  'micrometres',
  'to',
  '33.6',
  'metres',
  '.'],
 ['They',
  'have',
  'complex

In [87]:

# Flatten the list of words and remove punctuation and duplicates
flat_words = [word for sublist in words_lists for word in sublist]
filtered_words = [word.lower() for word in flat_words if word not in string.punctuation]
unique_words = list(Counter(filtered_words)) # We use here Counter, because it gives words that are in the same order as in the text, because while words are keys, their order in the text is the mapped value.
pprint(unique_words)

['animals',
 'are',
 'multicellular',
 'eukaryotic',
 'organisms',
 'that',
 'form',
 'the',
 'biological',
 'kingdom',
 'animalia',
 'with',
 'few',
 'exceptions',
 'consume',
 'organic',
 'material',
 'breathe',
 'oxygen',
 'able',
 'to',
 'move',
 'reproduce',
 'sexually',
 'and',
 'grow',
 'from',
 'a',
 'hollow',
 'sphere',
 'of',
 'cells',
 'blastula',
 'during',
 'embryonic',
 'development',
 'over',
 '1.5',
 'million',
 'living',
 'animal',
 'species',
 'have',
 'been',
 'described—of',
 'which',
 'around',
 '1',
 'insects—but',
 'it',
 'has',
 'estimated',
 'there',
 '7',
 'in',
 'total',
 'range',
 'size',
 '8.5',
 'micrometres',
 '33.6',
 'metres',
 'they',
 'complex',
 'interactions',
 'each',
 'other',
 'their',
 'environments',
 'forming',
 'intricate',
 'food',
 'webs',
 'study',
 'is',
 'called',
 'zoology']


In [83]:

# Calculate similarity scores and find matches
matches = []
not_in_vocab = []

threshold = 0.8
for word in unique_words:
    if word in wv:
        similarity = wv.similarity(word, keyword)
        if similarity > threshold:
            matches.append(word)
    else:
        not_in_vocab.append(word)

print(f"matches: {matches}", f"not_found: {not_in_vocab}", sep="\n")



matches: ['animal']
not_found: ['animalia', 'to', 'and', 'a', 'of', 'blastula', '1.5', 'described—of', 'insects—but', '8.5', '33.6', 'metres']


In [84]:
text_highlighted = text
for match in matches:
    text_highlighted = re.sub(r'\b' + match + r'\b', '**' + str.upper(match) + '**', text_highlighted, flags=re.IGNORECASE)


In [85]:
from IPython.display import display, Markdown
display(Markdown(text_highlighted))

Animals are multicellular eukaryotic organisms that form the biological kingdom Animalia. With few exceptions, animals consume organic material, breathe oxygen, are able to move, reproduce sexually, and grow from a hollow sphere of cells, the blastula, during embryonic development. Over 1.5 million living **ANIMAL** species have been described—of which around 1 million are insects—but it has been estimated there are over 7 million **ANIMAL** species in total. Animals range in size from 8.5 micrometres to 33.6 metres. They have complex interactions with each other and their environments, forming intricate food webs. The study of animals is called zoology.